# All Conference Transfers

In [1]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
from recordlinkage.index import Full
import queries

## Build out DataFrames

In [ ]:
limit = True
ranked = False

query = queries.get_query_all247(limit, ranked)
df_247 = (fx.connDBAndReturnDF(query)).set_index('IDYR')
df_247.index.name = '247_IDYR'


In [ ]:
limit = False
year = '2004'
query = queries.get_query_UnlinkedAllConference(limit,year)
df_ac = (fx.connDBAndReturnDF(query)).set_index('ID')
df_ac.index.name = 'AllConf_ID'

In [ ]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])

In [ ]:
#To avoid the Athlete issue, I'm not doing a full index instead of blocking
#indexer = recordlinkage.Index()
#indexer.add(Full())

In [ ]:
pairs = indexer.index(df_ac, df_247)

In [ ]:
from functions import YearOther

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearOther('Year', 'Year', label='Year'))
sumFields.append('Year')

In [ ]:
features = c.compute(pairs, df_ac, df_247)

In [ ]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['Sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

In [ ]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    #.929
    if (data['Sum'] > .8):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("fuzzy_results.csv")

In [ ]:
fx.create_AnnotationFile(dfFinal, df_ac, df_247)

In [2]:
filename = "..//Annotations//Results/result-ac.json"
keydataset = 4
targetkeydataset = 1
transfer = 0

fx.save_Annotations(filename, keydataset, targetkeydataset, transfer)

Completed. 11 records written to the database.
